In [1]:
import re
import linecache
from time import process_time
from collections import deque
import array as arr
from pathlib import Path


#Legge il file, estrae la matrice e la ritorna (lista di array)
def leggi_matrice(percorso_file):
    matrice = []
    with percorso_file.open('r') as file:
        for riga in file:
            if riga[0] != ";":
                nuova_riga = riga.replace("-", "")
                elementi = nuova_riga.split(" ")
                elementi.remove("\n")
                matrice.append(arr.array('B', list(map(int, elementi))))
    return matrice


def leggi_dominio(nome_file):
    riga_dominio = linecache.getline(nome_file, 5)
    dominio = re.findall('\(([^)]+)', riga_dominio)
    return dominio, riga_dominio


def carica_matrice(percorso_file):
    print(f'Caricamento matrice dal file {str(percorso_file)}')
    array = leggi_matrice(percorso_file)
    righe = len(array)
    colonne = len(array[0])
    nome_matrice = percorso_file.stem
    dominio_matrice, linea_dominio = leggi_dominio(str(percorso_file))
    if len(array[0]) == len(dominio_matrice):
        print('Matrice caricata con successo')
        print(f'Matrice: {nome_matrice}')
        print(f'Numero righe: {righe}')
        print(f'Numero colonne: {colonne}')
        dominio_base = arr.array('H', list(range(0, len(array[0]))))
        return nome_matrice, array, dominio_base, linea_dominio, True
    else:
        print('Errore sulle dimensioni del file')
        return nome_matrice, [], [], linea_dominio, False


#************************ ALG. BASE ***************************

#**************************************************************

#**************************************************************


def succ(elem):
    if elem == -1:
        return 1
    else:
        return elem + 1

def max_insieme(insieme):
    if not insieme:
        return -1
    else:
        return insieme[-1]


def alg_base(matrice, dominio):
    start_time = process_time()
    n_iter = 0
    lista_mhs = []
    coda = deque()
    valore_max = len(matrice[0])
    coda.append(arr.array('H', []))
    try:
        while coda:
            insieme = coda.popleft()
            m = max_insieme(insieme)
            e = succ(m)

            for elem in range(e, valore_max + 1):
                n_iter += 1
                nuovo_insieme = insieme[:]
                nuovo_insieme.append(elem)
                vett_rapp = crea_vett_rapp(nuovo_insieme, matrice)
                result = check(nuovo_insieme, vett_rapp)
                if result == "ok" and elem != valore_max:
                    coda.append(nuovo_insieme)
                elif result == "mhs":
                    lista_mhs.append(nuovo_insieme)
    except KeyboardInterrupt:
        print("Esecuzione terminata dall'utente\n")
        output(lista_mhs, dominio)
        return lista_mhs, process_time() - start_time, n_iter
    output(lista_mhs, dominio)
    return lista_mhs, process_time() - start_time, n_iter


def check(nuovo_insieme, vettore):
    proiezione = set(vettore) 
    for i in nuovo_insieme:
        if i not in proiezione:
            return "ko" 
 
    if 0 in proiezione:
        return "ok"
    else:
        return "mhs"  

def crea_vett_rapp(insieme, matrice):
    vett_rapp = arr.array('H', [0] * len(matrice))
    for i in range(len(matrice)):
        for j in insieme:
            if matrice[i][j - 1]:
                if vett_rapp[i]:
                    vett_rapp[i] = 65535
                    break
                else:
                    vett_rapp[i] = j
    return vett_rapp


# Funzione  che sostituisce i valori nei mhs calcolati (che sono compresi tra 1 e il numero di colonne della matrice)
def output(lista_mhs, dominio):
    for mhs in lista_mhs:
        for i in range(len(mhs)):
            mhs[i] = dominio[mhs[i] - 1]


#*****************************  PRE-ELABORAZIONE *********************************************

#********************************************************************************************

#********************************************************************************************


def contiene(matrice, i, j):
    for k in range(len(matrice[0])):
        if (not matrice[i][k]) and matrice[j][k]:
            return False
    return True


def costruisci_array(matrice):
    lista = []
    for i in range(len(matrice)):  # Numero di righe
        lista.append((sum(matrice[i]), i))
    lista.sort(key=lambda x: x[0], reverse=True)
    return arr.array('H', list(zip(*lista))[1])


def togli_righe(matrice):
    indici_rimossi = []
    array = costruisci_array(matrice)
    j = len(array)
    for i in range(j):
        for k in range(i + 1, j):
            if contiene(matrice, array[i], array[k]):
                indici_rimossi.append(array[i])
                break

    indici_rimossi.sort(reverse=True)
    for i in indici_rimossi:
        del matrice[i]
    indici_rimossi = arr.array('H', indici_rimossi)
    return indici_rimossi, len(indici_rimossi)


def colonna_di_zero(matrice, i):
    for j in range(len(matrice)):
        if matrice[j][i]:
            return False
    return True


def togli_colonne(matrice):
    indici_rimossi = []
    for i in range(len(matrice[0]) - 1, -1, -1):
        if colonna_di_zero(matrice, i):
            indici_rimossi.append(i)
    for i in indici_rimossi:
        deque(map(lambda x: x.pop(i), matrice), maxlen=0)
    indici_rimossi = arr.array('H', indici_rimossi)
    return indici_rimossi, len(indici_rimossi)


def pre_elaborazione_dominio(dominio, indici_da_rimuovere):
    if indici_da_rimuovere:
        for i in indici_da_rimuovere:
            del dominio[i]

################  ###########


def stringa_da_array(array, a_capo=True):
    if array:
        separatore = " "
        stringa = separatore.join([str(elem) for elem in array])
        if a_capo:
            return f'{stringa} -\n'
        else:
            return stringa
    else:
        return '\n'


def stampa_riepilogo(tempo, n_iterazioni, totale_mhs, min_mhs, max_mhs):
    print(f"Il tempo richiesto dall'esecuzione e' stato di {tempo} s")
    print(f"Il numero di iterazioni compiute e' stato di {n_iterazioni}")
    #print(f"La massima occupazione di memoria e' stata di {max_memoria} MiB")
    print(f"Sono stati trovati {totale_mhs} MHS")
    print(f"La cardinalita' minima dei MHS trovati e' {min_mhs}")
    print(f"La cardinalita' massima dei MHS trovati e' {max_mhs}\n")


def esegui_alg_base(percorso_file):
    _, matrice, dominio, _, esito = carica_matrice(percorso_file)
    if not esito:
        return 0
    else:
        soluzioni, tempo, n_iter = alg_base(matrice, dominio)
        stampa_riepilogo(tempo, n_iter, len(soluzioni), min(map(len, soluzioni)), max(map(len, soluzioni)))   
        return soluzioni


def alg_con_pre(matrice, dominio):
    start = process_time()
    righe_rimosse, numero_righe_rimosse = togli_righe(matrice)
    colonne_rimosse, numero_colonne_rimosse = togli_colonne(matrice)
    end = process_time()
    pre_elaborazione_dominio(dominio, colonne_rimosse)
    print(f"Il tempo richiesto dalla pre-elaborazione è stato di: {end - start} s")
    print("Numero righe rimosse: ", numero_righe_rimosse) 
    print("Numero colonne rimosse: ", numero_colonne_rimosse)
    print(f"Gli indici di riga rimossi sono: {stringa_da_array(righe_rimosse, a_capo=False)}")
    print(f"Gli indici di colonna rimossi sono: {stringa_da_array(sorted(colonne_rimosse), a_capo=False)}")
    lista_mhs, tempo_di_esecuzione, n_iter = alg_base(matrice, dominio)
    return lista_mhs, tempo_di_esecuzione, n_iter
    

def esegui_alg_pre(percorso_file):
    _, matrice, dominio, _, esito = carica_matrice(percorso_file)
    if not esito:
        return 0
    else:
        soluzioni, tempo, n_iter = alg_con_pre(matrice, dominio)
        stampa_riepilogo(tempo, n_iter, len(soluzioni), min(map(len, soluzioni)), max(map(len, soluzioni))) 
        return soluzioni


# **Esecuzione, caricare il file e incollare il suo percorso in "path"**








In [ ]:
# ESECUZIONE ALG. BASE
percorso_file = Path("/content/74283.010.matrix")

soluzioni = esegui_alg_base(percorso_file)

Caricamento matrice dal file /content/74283.010.matrix
Matrice caricata con successo
Matrice: 74283.010
Numero righe: 9
Numero colonne: 36
Il tempo richiesto dall'esecuzione e' stato di 0.09145345299999974 s
Il numero di iterazioni compiute e' stato di 11312
Sono stati trovati 247 MHS
La cardinalita' minima dei MHS trovati e' 2
La cardinalita' massima dei MHS trovati e' 4



In [2]:
# ESECUZIONE CON PRE-ELABORAZIONE
percorso_file = Path("/content/74181.014.matrix")

sol = esegui_alg_pre(percorso_file)

Caricamento matrice dal file /content/74181.014.matrix
Matrice caricata con successo
Matrice: 74181.014
Numero righe: 3
Numero colonne: 65
Il tempo richiesto dalla pre-elaborazione è stato di: 0.0002540579999998904 s
Numero righe rimosse:  0
Numero colonne rimosse:  44
Gli indici di riga rimossi sono: 

Gli indici di colonna rimossi sono: 1 3 4 5 6 7 8 9 13 14 15 16 17 18 19 21 22 23 25 26 29 30 33 34 35 36 37 38 39 40 41 42 43 48 49 50 53 54 55 56 58 59 60 62
Il tempo richiesto dall'esecuzione e' stato di 0.0017761890000000946 s
Il numero di iterazioni compiute e' stato di 291
Sono stati trovati 19 MHS
La cardinalita' minima dei MHS trovati e' 2
La cardinalita' massima dei MHS trovati e' 3

